<h1 style="text-align:center;">Mass Shootings in the USA</h1>

<p style="text-align:center;">Laia Mogas i Pau Mateo</p>
<p style="text-align:center;">Information Visualization - GCED</p>

## Obtaining data
The main data sources we used in this project are:
- https://www2.census.gov/
- https://www.gunviolencearchive.org/reports

From theese sources we obtain the following datasets:
- Mass shootings (january 2014 to september 2024)
- School incidents (january 2022 to 2024)
- County population


## Preprocessing and data cleaning - County Population
We obtained the conty popultion data with two separate csv files: one correspondig to the years 2014-2019, an another one for 2020 and later. These datasets contained a lot of information, but we only needed the population estimate foer each county every year, so we eliminated the unnecesary columns. We merged these two datasets with pytohn pandas. Then noticed that the state Connecticut had a county redistribuition in 2020, so we decided to keep the latest cofniguration of counties and estimate uniformly the population of those counties for years before 2020. The code we used for this preprocessing is the following:

In [56]:
import pandas as pd

pop2010_20 = pd.read_csv('datasets/co-est2010-2020_alldata.csv', encoding='latin1')
pop2020_23 = pd.read_csv('datasets/co-est2020-2023_alldata.csv', encoding='latin1')

########## column projection ##########
columns2010 = ['STATE', 'COUNTY','STNAME','CTYNAME']
columns2010.extend(['POPESTIMATE201'+str(i) for i in range(10)])
columns2020 = ['STATE', 'COUNTY','STNAME','CTYNAME']
columns2020.extend(['POPESTIMATE202'+str(i) for i in range(4)])

pop2010_20['FIPS'] = pop2010_20.apply(lambda row: f"{int(row['STATE']):02d}{int(row['COUNTY']):03d}", axis=1)
pop2020_23['FIPS'] = pop2020_23.apply(lambda row: f"{int(row['STATE']):02d}{int(row['COUNTY']):03d}", axis=1)


##########     merge     ##########
popComplete = pd.merge(pop2010_20[columns2010], pop2020_23[columns2020], on=['STATE', 'COUNTY'])

popComplete['STATE'] = popComplete['STATE'].astype(str)
popComplete['COUNTY'] = popComplete['COUNTY'].astype(str)


########## obraining FIPS ##########
popComplete['FIPS'] = popComplete.apply(lambda row: f"{int(row['STATE']):02d}{int(row['COUNTY']):03d}", axis=1)
popComplete['STNAME'] = popComplete['STNAME_x']
popComplete['CTYNAME'] = popComplete['CTYNAME_x']

########## ordering columns ##########
new_cols_order = ['STNAME', 'CTYNAME', 'FIPS', 'STATE', 'COUNTY'] + ['POPESTIMATE20'+f'{i:02d}' for i in range(14,24)]
popComplete = popComplete[new_cols_order]

In [57]:
##### checking changed counties #####

l1 = []
for c in pop2010_20['FIPS']:
    if c not in popComplete['FIPS'].values:
        l1.append(c)

l2 = []
for c in pop2020_23['FIPS']:
    if c not in popComplete['FIPS'].values:
        l2.append(c)

print(l1)
print(l2)

['09001', '09003', '09005', '09007', '09009', '09011', '09013', '09015']
['09110', '09120', '09130', '09140', '09150', '09160', '09170', '09180', '09190']


In [58]:
##### dealing with state '09' (connecticut) ######
from collections import defaultdict

fips_connecticut = pop2020_23[pop2020_23['STATE']==9]['FIPS'].values
N = len(fips_connecticut)

totalpop = defaultdict()


for i in range(14,24):
    year = 'POPESTIMATE20'+ f'{i:02d}'
    #calculate total population in state '09' in this year
    if i < 20:
        totalpop[year] = sum(pop2010_20[pop2010_20['STATE'] == 9][year].values)
    else:
        totalpop[year] = sum(pop2020_23[pop2020_23['STATE'] == 9][year].values)


for fips in fips_connecticut:
    new_row = pd.Series({
        'STNAME': 'Connecticut', 
        'CTYNAME': 'connecticut_county_unknown',
        'FIPS': fips, 
        'STATE': 9, 
        'COUNTY': fips[2:],
        'POPESTIMATE2014': totalpop['POPESTIMATE2014'] / N,
        'POPESTIMATE2015': totalpop['POPESTIMATE2015'] / N,
        'POPESTIMATE2016': totalpop['POPESTIMATE2016'] / N,
        'POPESTIMATE2017': totalpop['POPESTIMATE2017'] / N,
        'POPESTIMATE2018': totalpop['POPESTIMATE2018'] / N,
        'POPESTIMATE2019': totalpop['POPESTIMATE2019'] / N,
        'POPESTIMATE2020': totalpop['POPESTIMATE2020'] / N,
        'POPESTIMATE2021': totalpop['POPESTIMATE2021'] / N,
        'POPESTIMATE2022': totalpop['POPESTIMATE2022'] / N,
        'POPESTIMATE2023': totalpop['POPESTIMATE2023'] / N
    })
    popComplete.loc[len(popComplete)] = new_row


In [ ]:
#### save completed dataset ###
#  popComplete.to_csv('datasets/CountyPopulationAllYears.csv', index=False)

We'll then have to redistribute the Mass Shootings and School incidents that occured in the counties that we've just eliminated.

## Preprocessing and data cleaning - Mass Shootings + School incidents

To clean the Gun Violence data we used Open Refine followed by python.
#### Concatenating mass shootings csv
The raw data from the Gun Violence Archaive is splitted into various datasets. To add everything into a same csv file, we simply used Open Refine and opened all the csv when starting a new project. This way, the multiple csv are automatiqually concatenated, as they all have the same columns. Before doing that, we had to eliminate some of the duplicate rows in the data, as the file `MassShootings_2021.csv` overlaps with the file `GunVioleceAllYears.csv`. Again, we used Open Refine to do so, by making a time facet of the file `MassShootings_2021.csv` and selectig the rows that were already in the `GunVioleceAllYears.csv` file, and then eliminating them.

#### Tranformations
We applied some basic transformations to the dataset, mainly just changing the types of some columns, such as *Incident Date* to date, and numerical columns into intiger. 

#### Obtaining counties with OSM
To answer accuratelly the questions of the projecy, we needed the exact county where each incident occured. To obtain those, we first used Open Refine to obtain some json information from Open Street Maps, with the methos "Add column by fetching URLs". The command we used is:

With this, we obtained some json information for each row, for example:

In [ ]:
[{"place_id":323075484,
"licence":"Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
"osm_type":"relation",
"osm_id":1180533,
"lat":"38.6280278",
"lon":"-90.1910154",
"class":"boundary",
"type":"administrative",
"place_rank":12,
"importance":0.7078550940195126,
"addresstype":"independent_city",
"name":"Saint Louis",
"display_name":"Saint Louis, Missouri, United States",
"boundingbox":["38.5323215","38.7743018","-90.3206525","-90.1641941"]}]

From this information we wantet to obtain the county for each incident. However, we weren't capable of doing so with Open Refine, as the URL used didn't return the names of the counties in the same format as the dataest form census.gov, so we had to process again this informatino with pytohn.
We defined some functions that try to find a county name for each incident that exists in the population dataset obtained from census.gov. The functions iterate through the names in the key `"display_name"` from the json column (as there are some cases where the Township or some other information is also in the json code, so the county is not always in the same position) and applies some transformations to them and checks if the resulting name it's a county.

In [64]:
import pandas as pd
import json
from collections import defaultdict

pop = pd.read_csv("datasets/CountyPopulationAllYears.csv")

names_counties = defaultdict(set)  # names of the counties for each state

for state in pop['STNAME'].values:
    names_counties[state] = set(pop[pop['STNAME']==state]['CTYNAME'].values)


def transforms(name: str) -> str:
    if 'Clarke' in name: return 'Clarke County'
    if 'Washington' in name: return 'District of Columbia'
    if 'Connecticut' in name: return 'connecticut_county_unknown'
    name = name.replace('Saint', 'St.').replace('Nashville-', '').replace('Vista', 'Buena Vista').replace('Compton', 'Los Angeles County')
    name = name.replace('Monterey Park', 'Monterey County')
    return name.strip()


def try_find(name, state) -> str | None:
    '''tries to find the countie corresponding to `name` by 
    appliying transformations and trying diferent formats'''
    if name in names_counties[state]: return name

    if transforms(name) in names_counties[state]: return transforms(name)

    #try county
    n = transforms(name)
    if 'County' not in n: n += ' County'
    if n in names_counties[state]: return n

    #try parish
    n = transforms(name)
    if 'Parish' not in n: n += ' Parish'
    if n in names_counties[state]: return n

    #try city
    n = transforms(name)
    if 'city' not in n: n += ' city'
    if n in names_counties[state]: return n

    #try Municipality
    n = transforms(name)
    if 'city' not in n: n += ' Municipality'
    if n in names_counties[state]: return n
    
    else: return None


def extract_county(row) -> str | None:
    '''tries to extract the corresponding countie for the given row'''
    json_data = row['json']
    state = row['State'].strip()

    places = json.loads(json_data)
    if places:
        names = places[0].get("display_name", "")
        # Cerca "County", "Parish", o altres entitats equivalents en display_name
        
        delims = [",", ")", "("]

        for delim in delims:
            names = "_".join(names.split(delim))
        names = names.split("_")
        names = [n.strip() for n in names if n != '']

        for n in names:
            cname = try_find(n, state)
            if cname: return cname
            else: continue
    
        return None
    

SchoolIncidents = pd.read_csv('datasets/Schoolincidents-json.csv')
MassShootingsComplete = pd.read_csv("datasets/MassShootingsComplete-json.csv")

MassShootingsComplete['county'] = MassShootingsComplete.apply(extract_county, axis=1)
SchoolIncidents['county'] = SchoolIncidents.apply(extract_county, axis=1)

#remove rows with null or unknown county
MassShootingsComplete = MassShootingsComplete.dropna(subset=['county'])
SchoolIncidents = SchoolIncidents.dropna(subset=['county'])

With this transformations we obtain a county for **all rows** that had a not null value in the json column (for 70 rows amongst 5k, the URLs from Open Refine extracted just `"[]"`).

### Obtaining FIPS codes

Obtaining the FIPS codes for each county facilitated us working with altair's cloropleth charts. At this point, it was easy to obtain the FIPS codes for each incident, as we already had them in the County population csv. A simple function was enough:

In [ ]:
from random import choice

def get_fips(row) -> str | None:
    """
    Returns the FIPS code for the given row by searching
    it in the population csv
    """

    county_value = row['county']

    # connecticut separate case:
    connecticut_fips = ['09110', '09120', '09130', '09140', '09150', '09160', '09170', '09180', '09190']
    if county_value == 'connecticut_county_unknown':
        fips = choice(connecticut_fips)  # we map the incident into a random county
        return f'{int(fips):02d}'

    if county_value is not None:
        county_value = county_value.strip()
    else:
        county_value = ""  # Si és None, assignar una cadena buida per evitar errors
    
    fips_value = pop[(pop['STNAME'] == row['State']) & (pop['CTYNAME'] == county_value)]['FIPS']
    
    # Retornar el primer valor (ja que és únic)
    return f'{fips_value.iloc[0]:02d}' if not fips_value.empty else None


MassShootingsComplete['FIPS'] = MassShootingsComplete.apply(get_fips, axis=1)
MassShootingsComplete['FIPS'] = MassShootingsComplete['FIPS'].apply(lambda x: f"{int(x):05d}") # to ensure length 5


SchoolIncidents['FIPS'] = SchoolIncidents.apply(get_fips, axis=1)
SchoolIncidents['FIPS'] = SchoolIncidents['FIPS'].apply(lambda x: f"{int(x):05d}")

The preprocessing is done! We are ready to start making visualizations

## Questions

### **Q1**: What are the states with large number of mass shootings per citizen?

### **Q2:** How is the number of mass shootings per citizen distributed across the different counties in the US? And across states?

### **Q3:** Are the mass shootings correlated with gun violence incidents in schools?

### **Q4:** How have mass shootings evolved the last years in the US?